In [1]:
import os
import copy
import shutil
import pandas as pd

In [2]:
# set path
# first copy original files into fmt_path
raw_path = "D:\\EGI_DATA\\EGI_RAW"
fmt_path = "D:\\EGI_DATA\\EGI_FMT"
edf_path = "D:\\EGI_DATA\\EGI_EDF"
bids_root_path = "D:\\EGI_DATA\\EGI_BIDS"
beh_path = "beh"
rec_path = "rec"
eeg_path = "eeg"
# set task name
wordtask = "wordchoice"
imagetask = "imagechoice"
foodtask = "foodchoice"
rest = "rest"

# 1. format raw data

## delete screen recording

In [3]:
for sub in os.listdir(fmt_path):
    for direc in os.listdir(os.path.join(fmt_path, sub)):
        if os.path.exists(os.path.join(fmt_path, sub, "Screencapture")):
            shutil.rmtree(os.path.join(fmt_path, sub, "Screencapture"))

## rename "EEG DATA"、"Behavioral Data"

In [4]:
## TODO: generate data from raw_path instead of fmt_path
## rename directory name under "sub-xxx"
for sub in os.listdir(fmt_path):
    # under "sub-xxx" directory
    for directory in os.listdir(os.path.join(fmt_path, sub)):
        # eeg data
        if "EEG DATA" in directory or "EEG_DATA" in directory:
            # print(os.path.join(fmt_path, sub, directory))
            os.rename(os.path.join(fmt_path, sub, directory), os.path.join(fmt_path, sub, eeg_path))
        # behavioral data
        elif "Beha" in directory:
            # print(os.path.join(fmt_path, sub, directory))
            os.rename(os.path.join(fmt_path, sub, directory), os.path.join(fmt_path, sub, beh_path))

## behavioral data

In [5]:
## for "beh" dir
task_set = set(["foodEnd", "foodChoice", "foodHealthy", "imageChoice", "wordChoice", "wordImageMatch", "foodTaste"])
for sub in os.listdir(fmt_path):
    # under "sub-xxx" directory
    # count = 0
    # total = 0
    for csv in os.listdir(os.path.join(fmt_path, sub, beh_path)):
        # under "sub-xxx/beh" directory
        # total += 1
        cross = set(csv.split("_")) & task_set
        if csv not in task_set and len(cross) != 0:
            # count += 1
            # print(os.path.join(fmt_path, sub, beh_path, csv))
            os.rename(os.path.join(fmt_path, sub, beh_path, csv), os.path.join(fmt_path, sub, beh_path, "{}_task-{}_beh.csv".format(sub, list(cross)[0]).lower()))
    # print("{}: {}/{}".format(os.path.join(fmt_path, sub, beh_path), count, total))

## eeg data

In [6]:
task_dict = {"-F": "foodchoice", "FC_": "foodchoice", "-I": "imagechoice", "IC_": "imagechoice", "-W": "wordchoice", "WC_": "wordchoice", "-R": "rest", "RS_": "rest", "-WM": "wordimagematch", "WM_": "wordimagematch"}
for sub in os.listdir(fmt_path):
    # sub-001 to sub-015
    if sub <= "sub-015":
        if not os.path.exists(os.path.join(fmt_path, sub, eeg_path, "ori_mff")):
            continue
        for mff in os.listdir(os.path.join(fmt_path, sub, eeg_path, "ori_mff")):
            if "fil" in mff:
                continue
            # print(os.path.join(fmt_path, sub, eeg_path, "ori_mff", mff))
            for key, value in task_dict.items():
                if key in mff.upper():
                    shutil.move(os.path.join(fmt_path, sub, eeg_path, "ori_mff", mff), os.path.join(fmt_path, sub, eeg_path, "{}_task-{}_eeg.mff".format(sub, value)))
                    break
        # remove filtered files
        shutil.rmtree(os.path.join(fmt_path, sub, eeg_path, "ori_mff"))
        shutil.rmtree(os.path.join(fmt_path, sub, eeg_path, "fil_edf"))
    # sub-016 to sub-031
    else:
        for mff in os.listdir(os.path.join(fmt_path, sub, eeg_path)):
            # delete filtered edf data
            if "fil.edf" in mff:
                os.remove(os.path.join(fmt_path, sub, eeg_path, mff))
            # delete filtered mff data
            elif "fil.mff" in mff:
                shutil.rmtree(os.path.join(fmt_path, sub, eeg_path, mff))
            else:
                for key, value in task_dict.items():
                    if key in mff.upper():
                        os.rename(os.path.join(fmt_path, sub, eeg_path, mff), os.path.join(fmt_path, sub, eeg_path, "{}_task-{}_eeg.mff".format(sub, value)))
                        break

## delete behavioral data and wordimagematch eeg data

In [7]:
for sub in os.listdir(fmt_path):
    shutil.rmtree(os.path.join(fmt_path, sub, beh_path), ignore_errors=True)
    shutil.rmtree(os.path.join(fmt_path, sub, eeg_path, "{}_task-{}_eeg.mff".format(sub, "wordimagematch")), ignore_errors=True)

# 2. convert .mff file to .edf file

## convert mff to edf

In [3]:
from save_edf import write_mne_edf

In [6]:
if os.path.exists(edf_path):
    shutil.rmtree(edf_path)
else:
    os.mkdir(edf_path)

In [8]:
# place edf files in edf_path
for sub in os.listdir(fmt_path):
    # create sub/eeg path if not exist
    if not os.path.exists(os.path.join(edf_path, sub)):
        os.makedirs(os.path.join(edf_path, sub, eeg_path))
    for mff in os.listdir(os.path.join(fmt_path, sub, eeg_path)):
        edf_file_path = os.path.join(edf_path, sub, eeg_path, "{}edf".format(mff[:-3]))
        if not os.path.exists(edf_file_path):
            write_mne_edf(os.path.join(fmt_path, sub, eeg_path, mff), edf_file_path, overwrite=True)

Reading EGI MFF Header from D:\EGI_DATA\EGI_FMT\sub-020\eeg\sub-020_task-imagechoice_eeg.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {0000} ...
Reading 0 ... 1652992  =      0.000 ...  1652.992 secs...
saving to D:\EGI_DATA\EGI_EDF\sub-020\eeg\sub-020_task-imagechoice_eeg.edf, filetype 1
959 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 

# 3. BIDS

## convert to bids

In [9]:
import mne
from mne_bids import write_raw_bids, BIDSPath, print_dir_tree

In [10]:
if os.path.exists(bids_root_path):
    shutil.rmtree(bids_root_path)
else:
    os.makedirs(bids_root_path)

In [31]:
# TODO
for sub in os.listdir(edf_path):
    # create sub/eeg path if not exist
    # if not os.path.exists(os.path.join(edf_path, sub)):
    #     os.makedirs(os.path.join(edf_path, sub, eeg_path))
    for edf_file in os.listdir(os.path.join(edf_path, sub, eeg_path)):
        subject_id = edf_file.split("_")[0].split("-")[1]
        task = edf_file.split("_")[1].split("-")[1]
        bids_path = BIDSPath(subject=subject_id, task=task, root=bids_root_path)
        raw = mne.io.read_raw_edf(os.path.join(edf_path, sub, eeg_path, edf_file), preload=False)
        raw.info["line_freq"] = 50
        write_raw_bids(raw, bids_path, overwrite=True)

NameError: name 'mne' is not defined

In [ ]:
# sub = "sub-021"
# bids_path = BIDSPath(subject="021", task=wordtask, root=bids_root_path)
# raw = mne.io.read_raw_edf(os.path.join(fmt_path, sub, eeg_path, "{}_task-{}_eeg.edf".format(sub, wordtask)), preload=False)
# raw.info["line_freq"] = 50
# write_raw_bids(raw, bids_path, overwrite=True)

In [ ]:
# for sub in os.listdir(fmt_path):
#     for mff in os.listdir(os.path.join(fmt_path, sub, eeg_path)):
#         mff_path = os.path.join(fmt_path, sub, eeg_path, mff)
#         if not os.path.exists(mff_path[:-3] + "edf"):
#             write_mne_edf(mff_path, mff_path[:-3] + "edf", overwrite=True)